In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-dataset/Titanic-Dataset.csv


In [2]:
# Read the data into a dataframe where we can do the rest of our transformations (file path was found by running ^^ cell)
data = pd.read_csv('/kaggle/input/titanic-dataset/Titanic-Dataset.csv')

In [3]:
# Let's get a peek of what the data looks like
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# For the numeric values let's get some quick stats
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
# Were the upper socio economic class tickets associated with more preferred treatment loading the lifeboats?
high_fare_cutoff = 100
data['high_fare'] = data['Fare'].apply(lambda x: True if x > high_fare_cutoff else False)
pd.crosstab(index=data['high_fare'], columns=data['Survived'], values=data['PassengerId'], aggfunc='count', normalize=True)

Survived,0,1
high_fare,,
False,0.600449,0.340067
True,0.015713,0.043771


In [6]:
# "Women and children first"
data['is_female'] = data['Sex'].apply(lambda x: True if x == 'female' else False)
pd.crosstab(index=data['is_female'], columns=data['Survived'], values=data['PassengerId'], aggfunc='count', normalize=True)

Survived,0,1
is_female,,
False,0.525253,0.122334
True,0.090909,0.261504


In [7]:
def predict(row):
    if row['is_female']:
        if row['high_fare']:
            return 1
        else:
            return 0
    else:
        return 0

data['pred'] = data.apply(lambda x: predict(x), axis=1)

# This will show percentages in each quadrant (TN, FN, FP, TP)
print('Confusion matrices:')
print('\n')
print('counts:')
print(pd.crosstab(data['Survived'], data['pred'], values=data['PassengerId'], aggfunc='count'))
print('\n')
print('percentages:')
print(pd.crosstab(data['Survived'], data['pred'], values=data['PassengerId'], aggfunc='count', normalize=True))
print('\n')


def print_metrics(data):
    true_negative_count = len(data[(data['Survived'] == 0) & (data['pred'] == 0)])
    false_negative_count = len(data[(data['Survived'] == 1) & (data['pred'] == 0)])
    false_positive_count = len(data[(data['Survived'] == 0) & (data['pred'] == 1)])
    true_positive_count = len(data[(data['Survived'] == 1) & (data['pred'] == 1)])
    total_count = len(data)
    
    accuracy = (true_positive_count + true_negative_count) / total_count
    precision = true_positive_count / (true_positive_count + false_positive_count)
    recall = true_positive_count / (true_negative_count + false_positive_count)
    
    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)

print('Metrics:')
print_metrics(data)

Confusion matrices:


counts:
pred        0   1
Survived         
0         547   2
1         310  32


percentages:
pred             0         1
Survived                    
0         0.613917  0.002245
1         0.347924  0.035915


Metrics:
Accuracy: 0.6498316498316499
Precision: 0.9411764705882353
Recall: 0.058287795992714025
